In [4]:
import gymnasium as gym

In [5]:
class GridWorldEnv(gym.Env):
    def __init__(self, size: int = 5):
        # The size of the square grid
        self.size = size

        # Define the agent and target location;
        # randomly chosen in `reset` and update in `step`
        self._agent_location = np.array([-1, -1], dtype=np.int32)
        self._agent_location = np.array([-1, -1], dtype=np.int32)

        # Observation are dictionaries with the agent's and the target's location.
        # Each location is encoded as an element of {0,...,`size`-1}^2
        self.observation_space = gym.space.Dict(
            {
                "agent": gym.spaces.Box(0, size-1, shape=(2,), dtype=int),
                "target": gym.spaces.Box(0, size-1, shape(2,), dtype=int),
            }
        )

        # We have 4 actions, corresponding to "right", "up", "left", "down"
        self.action_space = gym.spaces.Discrete(4)
        # Dictionary maps the abstract actions to the directions on the grid
        self._action_to_direction = {
            0: np.array([1, 0]),  # right
            1: np.array([0, 1]),  # up
            2: np.array([-1, 0]), # left
            3: np.array([0, -1]), # down
        }

    def _get_obs(self):
        return {"agent": self._agent_location, "target": self._target_location}

    def _get_info(self):
        return {"distance": np.linalg.norm(self._agent_location - self._target_location, ord=1)}
